# Smart Scheduling
**Goal**: Use existing patterns in attendance and academic performance to simulate alternative scheduling strategies (e.g., shifting certain courses, grouping high-engagement sessions earlier/later) to optimize expected attendance.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

schedule_df = pd.read_csv('schedule_data.csv')
attendance_df = pd.read_csv('attendance_feie.csv')

# Clean column names
attendance_df.columns = (
    attendance_df.columns.str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('à', 'a')
    .str.replace('è', 'e')
    .str.replace('é', 'e')
    .str.replace('ò', 'o')
    .str.replace('ç', 'c')
    .str.replace('ñ', 'n')
    .str.replace('á', 'a')
    .str.replace('í', 'i')
    .str.replace('ó', 'o')
    .str.replace('ú', 'u')
)

In [ ]:
# Create group label
attendance_df['grup_estudi'] = attendance_df['titulacio'] + " - " + attendance_df['curs'].astype(str) + " - " + attendance_df['horari']
schedule_df['grup_estudi'] = schedule_df['titulacio'] + " - " + schedule_df['curs'].astype(str) + " - " + schedule_df['horari']

day_mapping = {
    1: 'dilluns',
    2: 'dimarts', 
    3: 'dimecres',
    4: 'dijous',
    5: 'divendres'
}

# Session-level attendance rates
session_cols = [col for col in attendance_df.columns if col.startswith('s') and col[1:].isdigit()]

attendance_df['attendance_total'] = attendance_df[session_cols].sum(axis=1)
attendance_df['attendance_rate'] = (
    attendance_df['attendance_total'] /
    (attendance_df['num_sessions'] * attendance_df['matricules-assignatura'])
)
attendance_df['engagement_score'] = (0.5 * attendance_df['attendance_rate'] + 0.5 * attendance_df['taxa_de_rendiment'])

ade_1_mati = attendance_df[attendance_df['grup_estudi'].str.contains("ADE - 1 - mati")]
summary = ade_1_mati[['assignatura', 'attendance_rate', 'taxa_de_rendiment', 'engagement_score']].sort_values(by='engagement_score', ascending=False)
print(summary)

                        assignatura  attendance_rate  taxa_de_rendiment  \
118                 Matemàtiques II         0.673333              0.844   
51                    Estadística I         0.615385              0.890   
89   Introducció a la comptabilitat         0.675000              0.742   
40           Economia Internacional         0.520833              0.896   

     engagement_score  
118          0.758667  
51           0.752692  
89           0.708500  
40           0.708417  


In [ ]:
# Process schedule data to extract temporal features
def extract_time_features(df):
    def normalize_time(time_str):
        if '.' not in str(time_str): 
            return f"{time_str}.00"
        return str(time_str)
    
    # Extract start and end times
    time_parts = df['hour'].str.split('-', expand=True)
    df['start_time'] = time_parts[0].apply(normalize_time)
    df['end_time'] = time_parts[1].apply(normalize_time)
    
    # Convert to datetime.time objects
    df['start_datetime'] = pd.to_datetime(df['start_time'], format='%H.%M', errors='coerce').dt.time
    df['end_datetime'] = pd.to_datetime(df['end_time'], format='%H.%M', errors='coerce').dt.time
    
    df['duration'] = (
        pd.to_datetime(df['end_time'], format='%H.%M') - 
        pd.to_datetime(df['start_time'], format='%H.%M')
    ).dt.total_seconds() / 3600
    
    # names to lowercase to match attendance data
    df['day'] = df['day'].str.lower()
    
    # Session type (0=lecture, 1=practical)
    df['session_type'] = df['tipus'].map({0: 'lecture', 1: 'practical'})
    
    return df

schedule_df = extract_time_features(schedule_df)
print(schedule_df.head())

  titulacio  curs horari           assignatura  codi_assignatura  semestre  \
0       ADE     1   mati        Matemàtiques I            102345         1   
1       ADE     1   mati        Matemàtiques I            102345         1   
2       ADE     1   mati                  Dret            102340         1   
3       ADE     1   mati                  Dret            102340         1   
4       ADE     1   mati  Hª Econòmica Mundial            102330         1   

        day  day_code  session         hour  tipus     grup_estudi start_time  \
0   dilluns         1        1         9-11      0  ADE - 1 - mati       9.00   
1  dimecres         3        2  11.15-13.15      1  ADE - 1 - mati      11.15   
2   dimarts         2        1         9-11      0  ADE - 1 - mati       9.00   
3    dijous         4        2  11.15-14.15      1  ADE - 1 - mati      11.15   
4  dimecres         3        1         9-11      0  ADE - 1 - mati       9.00   

  end_time start_datetime end_datetime  dura

**Merge Schedule and Attendance Data**

In [ ]:
merged_df = pd.merge(
    schedule_df,
    attendance_df,
    left_on=['assignatura', 'codi_assignatura', 'grup_estudi'],
    right_on=['assignatura', 'codi_assignatura', 'grup_estudi'],
    how='left'
)

**Additional Features**

In [ ]:
def create_temporal_features(df):
    # Time of day categories
    df['time_of_day'] = pd.cut(
        pd.to_datetime(df['start_time'], format='%H.%M').dt.hour,
        bins=[0, 10, 13, 17, 24],
        labels=['early_morning', 'late_morning', 'afternoon', 'evening'],
        right=False
    )
    return df

merged_df = create_temporal_features(merged_df)

In [23]:
features_to_keep = [
    'grup_estudi', 'assignatura', 'codi_assignatura', 'semestre',
    'day', 'day_code', 'session', 'hour', 'start_time', 'end_time',
    'duration', 'session_type', 'matricules-assignatura',
    'attendance_rate', 'taxa_de_rendiment', 'engagement_score', 'time_of_day'
]

final_df = merged_df[features_to_keep]

# 12. Save prepared data
final_df.to_csv('prepared_schedule_data.csv', index=False)